In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

import multiprocessing as mp
import pickle 
import warnings 
warnings.filterwarnings('ignore')

from imports import*
from utils import *
from logistic_regression import *
from rnn import *

from hybrid_sim import *
from hybrid_fit import *
from hybrid_predict import *

from habit_sim import *
from habit_fit import *
from habit_predict import *

from kdh_sim import *
from kdh_fit import *
from kdh_predict import *

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# num of agent
num_of_agents = 200 # 200

# num of block
num_of_block = 2

# num of trials 
num_of_trials = 200

# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)

models = {

    'hybrid':[ configuration_parameters_hybrid,
               hybrid_sim,
               hybrid_fit,
               hybrid_predict],
    
    'habit':[  configuration_parameters_habit,
               habit_sim,
               habit_fit,
               habit_predict],
    
    'kdh':  [  configuration_parameters_kdh,
               kdh_sim,
               kdh_fit,
               kdh_predict]
}


def bce(y_hat,y_true):
    eps = 1e-7
    return -np.sum( y_true*np.log(y_hat+eps) + (1-y_true)*np.log(1-y_hat+eps) )

In [ ]:
# sim
for m in models:
    print(f'Model {m}')
    
    data_per_agent = []
    parameters = []

    for agent in tqdm(range(num_of_agents)):
        param = models[m][0]()
        parameters.append(param)
        
        data = []
        for i in range(num_of_block):
            # create rewards probs 
            reward_probs = create_rndwlk(4,num_of_trials)
            df = models[m][1](
                            param,
                            num_of_trials,
                            reward_probs
            ) 
            data.append(df)
        data_per_agent.append(data)

    df = pd.DataFrame(parameters)
    
    for agent in range(num_of_agents):
        for block in range(num_of_block):
            data_per_agent[agent][block].to_csv(f'../data_finetune/{m}/{m}_agent_{agent}_sim_{block}.csv',index=False)
    

In [ ]:
# upload data
all_data = [] 
for sim in range(num_of_block):
    data_per_block = []
    for m in tqdm(models): 
        for agent in range(num_of_agents):
            data_per_block.append((pd.read_csv(f'../data_finetune/{m}/{m}_agent_{agent}_sim_{sim}.csv')))
    all_data.append(data_per_block)
    
block_0 = all_data[0]
block_1 = all_data[1]

random.shuffle(block_0)
random.shuffle(block_1)

all_blocks = [block_0,block_1]

In [ ]:
def train_model_pre(net, train_loader, val_loader , epochs, lr):
        
    train_loss, train_ll = np.zeros(epochs), np.zeros(epochs)
    val_loss ,val_ll = np.zeros(epochs), np.zeros(epochs)
    min_loss_v = 100

    # move net to GPU
    net.to(device)
    # Use Adam optimizer
    optimizer = optim.Adam(net.parameters(), lr=lr) 
    criterion = nn.BCELoss()
    
    # Loop over epochs
    for i in tqdm(range(epochs)):
        
        # Loop over training batches
        running_loss_tr = []
        for j,(X_train,y_train) in enumerate(train_loader):
            
            # move to GPU
            X_train , y_train = X_train.to(device), y_train.to(device)
            # reshape to 1 X batch_size X input_size
            X_train = X_train.reshape(1,X_train.shape[0], INPUT_SIZE)
            # zero the gradient buffers
            optimizer.zero_grad() 
            out, hn = net(X_train)
            # Reshape to (SeqLen x Batch, OutputSize)
            out = out.view(-1, OUTPUT_SIZE)
            loss = criterion(out, y_train)
            loss.backward()
            optimizer.step() # Does the update
            running_loss_tr.append(loss.item())
            
        train_loss[i], train_ll[i] = eval_net(net, train_loader)
        val_loss[i], val_ll[i] = eval_net(net, val_loader)
        
        if val_loss[i] <= min_loss_v:
            checkpoint = {'epoch':i+1,'model_state':net.state_dict(),
                          'optim_state':optimizer.state_dict(),'loss':val_loss[i]}
            torch.save(checkpoint,f'rnn_finetune_5_200.pth')
            min_loss_v = val_loss[i]
        
        print('train loss: ', train_loss[i])
        print('val loss: ', val_loss[i])
        net.train()

    return net, train_loss , train_ll , val_loss, val_ll

In [ ]:
df_train = pd.concat(all_blocks[0]).reset_index()
df_val = pd.concat(all_blocks[1]).reset_index()

INPUT_SIZE = 5
OUTPUT_SIZE = 2
LERANING_RATE = 0.001

hidden_size = 5
num_layers = 1
epochs = 500

train_data = behavior_dataset(df_train)
val_data = behavior_dataset(df_val)

train_loader = DataLoader(train_data,shuffle=False, batch_size=1_000)
val_loader = DataLoader(val_data,shuffle=False, batch_size=1_000)

rnn = GRU_NN(INPUT_SIZE, hidden_size, num_layers, OUTPUT_SIZE)
rnn, train_loss, train_ll, val_loss, val_ll, = train_model_pre(rnn,
                                                            train_loader,
                                                            val_loader,
                                                            epochs=epochs,
                                                            lr=LERANING_RATE) 



plt.plot(train_loss)
plt.plot(val_loss)

In [ ]:
# num of agent
num_of_agents = 100

# num of block
num_of_block = 3

# num of trials 
num_of_trials = 200

# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)

all_data = [] 
for sim in range(num_of_block):
    data_per_block = []
    for m in tqdm(models): 
        for agent in range(num_of_agents):
            data_per_block.append((pd.read_csv(f'../data/{m}/{m}_agent_{agent}_sim_{sim}.csv')))
    all_data.append(data_per_block)
    
block_0 = all_data[0]
block_1 = all_data[1]
block_2 = all_data[2]

all_blocks = [block_0,block_1,block_2]

In [ ]:
N = num_of_agents*3 # 3 models

INPUT_SIZE = 5
OUTPUT_SIZE = 2
LERANING_RATE = 0.001

hidden_size = 5
num_layers = 1
epochs = 200

loss_train, loss_val, loss_test  = [], [], []
ll_train, ll_val, ll_test = [], [], []

for n in tqdm(range(N)):
    for train, val, test in cv:

        train_data = behavior_dataset(all_blocks[train][n])
        val_data = behavior_dataset(all_blocks[val][n])
        test_data = behavior_dataset(all_blocks[test][n])

        train_loader = DataLoader(train_data,shuffle=False,batch_size=len(train_data))
        val_loader = DataLoader(val_data,shuffle=False,batch_size=len(val_data))
        test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))
        
        load = torch.load(f'rnn_finetune_5_200.pth')
        model = GRU_NN(INPUT_SIZE, hidden_size, num_layers, OUTPUT_SIZE)
        model.load_state_dict(load['model_state'])
        
        _, train_loss, train_ll, val_loss, val_ll, test_loss, test_ll = train_model(model,
                                                                                train_loader,
                                                                                val_loader,
                                                                                test_loader,
                                                                                epochs=epochs,
                                                                                lr=LERANING_RATE) 
                                                                                                                                       
        loss_train.append(train_loss)
        loss_val.append(val_loss)
        loss_test.append(test_loss)
        
        ll_train.append(train_ll)
        ll_val.append(val_ll)
        ll_test.append(test_ll)
        
    print('Done agent',n)
    
    
with open('../results_finetune/loss_train.pickle', 'wb') as handle:
    pickle.dump(loss_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/loss_val.pickle', 'wb') as handle:
    pickle.dump(loss_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/loss_test.pickle', 'wb') as handle:
    pickle.dump(loss_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/ll_train.pickle', 'wb') as handle:
    pickle.dump(ll_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../results_finetune/ll_val.pickle', 'wb') as handle:
    pickle.dump(ll_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/ll_test.pickle', 'wb') as handle:
    pickle.dump(ll_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
        